In [ ]:
%pip install rembg
%pip install google
%pip install onnxruntime
%pip install keras
%pip install tensorflow

In [ ]:
import pandas as pd
from rembg import remove
import cv2
import os
import numpy as np
from keras.utils import to_categorical, plot_model
from keras.models import Sequential, load_model
from keras.layers import Dropout, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Softmax
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam

In [ ]:
import os, random, numpy as np
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["TF_DETERMINISTIC_OPS"] = "1"

import tensorflow as tf
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)
try:
    tf.config.experimental.enable_op_determinism()
except Exception:
    pass

In [1]:
from pathlib import Path
import zipfile
import ipynbname

nb_path = Path(str(ipynbname.path()))
FOLDER_ROOT = nb_path.parent.resolve()
PROJECT_ROOT = FOLDER_ROOT.parent.resolve()
print("FOLDER_ROOT:", FOLDER_ROOT)

SYNTH_ROOT = PROJECT_ROOT / "synthetic_dataset"
SYNTH_ZIP  = PROJECT_ROOT / "synthetic_dataset.zip"

MODELS_DIR  = PROJECT_ROOT / "models"
REPORTS_DIR = PROJECT_ROOT / "reports"
MODELS_DIR.mkdir(parents=True, exist_ok=True)
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

if not SYNTH_ROOT.exists() and SYNTH_ZIP.exists():
    with zipfile.ZipFile(SYNTH_ZIP, 'r') as zf:
        zf.extractall(PROJECT_ROOT)
    print(f"Descomprimido: {SYNTH_ZIP.name} -> {SYNTH_ROOT}")
assert (SYNTH_ROOT / "train").exists(), "Falta synthetic_dataset/train. Genera o descomprime el dataset sintético."
assert (SYNTH_ROOT / "val").exists(),   "Falta synthetic_dataset/val."
assert (SYNTH_ROOT / "test").exists(),  "Falta synthetic_dataset/test."

TRAIN_DIR = SYNTH_ROOT / "train"
VAL_DIR   = SYNTH_ROOT / "val"
TEST_DIR  = SYNTH_ROOT / "test"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("TRAIN_DIR:", TRAIN_DIR)
print("VAL_DIR:  ", VAL_DIR)
print("TEST_DIR: ", TEST_DIR)
print("MODELS_DIR:", MODELS_DIR)


FOLDER_ROOT: D:\tesis\para-articulo\malnutrition-reproducibility\notebooks
Descomprimido: synthetic_dataset.zip -> D:\tesis\para-articulo\malnutrition-reproducibility\synthetic_dataset
PROJECT_ROOT: D:\tesis\para-articulo\malnutrition-reproducibility
TRAIN_DIR: D:\tesis\para-articulo\malnutrition-reproducibility\synthetic_dataset\train
VAL_DIR:   D:\tesis\para-articulo\malnutrition-reproducibility\synthetic_dataset\val
TEST_DIR:  D:\tesis\para-articulo\malnutrition-reproducibility\synthetic_dataset\test
MODELS_DIR: D:\tesis\para-articulo\malnutrition-reproducibility\models


In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score


class_names = ['Peso adecuado', 'Aguda', 'Moderada', 'Severa']

def _to_index(y):
    """
    Convierte etiquetas a índices 0..C-1.
    Acepta one-hot o índices; devuelve shape (N,).
    """
    y = np.asarray(y)
    if y.ndim > 1 and y.shape[-1] == len(class_names):
        return y.argmax(axis=1)
    return y.astype(int)

def evaluate_model_text(model, X, y, model_name='Modelo'):
    """
    Devuelve (metrics_dict, text_report)
      - metrics_dict: {'model','cm','cm_df','f1_per_class','f1_macro','f1_weighted','auc_per_class','auc_macro'}
      - text_report: string con F1 y AUC en formato legible (sin gráficos)
    """
    
    y_true_idx = _to_index(y)            
    y_prob = model.predict(X, verbose=0) 
    y_pred_idx = np.argmax(y_prob, axis=1)

    
    labels_idx = list(range(len(class_names)))
    cm = confusion_matrix(y_true_idx, y_pred_idx, labels=labels_idx)
    cm_df = pd.DataFrame(
        cm,
        index=pd.Index(class_names, name='Real'),
        columns=pd.Index(class_names, name='Predicción')
    )

    
    f1_per_class = f1_score(y_true_idx, y_pred_idx, average=None, labels=labels_idx)
    f1_macro = f1_score(y_true_idx, y_pred_idx, average='macro')
    f1_weighted = f1_score(y_true_idx, y_pred_idx, average='weighted')

    
    try:
        y_true_oh = tf.keras.utils.to_categorical(y_true_idx, num_classes=len(class_names))
        auc_per_class = roc_auc_score(y_true_oh, y_prob, average=None, multi_class='ovr')
        auc_macro = roc_auc_score(y_true_oh, y_prob, average='macro', multi_class='ovr')
        auc_note = None
    except Exception as e:
        
        auc_per_class = np.array([np.nan] * len(class_names))
        auc_macro = np.nan
        auc_note = f"AUC no computable (¿clase ausente en test?). Detalle: {e}"

    
    lines = [f"== {model_name} ==", "F1 por clase:"]
    for i, name in enumerate(class_names):
        lines.append(f"  {name}: {f1_per_class[i]:.4f}")
    lines.append(f"F1 macro: {f1_macro:.4f} | F1 weighted: {f1_weighted:.4f}")
    lines.append("")
    if np.isnan(auc_macro):
        lines.append("AUC (ROC): " + (auc_note or "no computable."))
    else:
        lines.append("AUC (ROC) por clase:")
        for i, name in enumerate(class_names):
            lines.append(f"  {name}: {auc_per_class[i]:.4f}")
        lines.append(f"AUC macro: {auc_macro:.4f}")
    text_report = "\n".join(lines)

    metrics = {
        'model': model_name,
        'cm': cm,               
        'cm_df': cm_df,         
        'f1_per_class': f1_per_class,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'auc_per_class': auc_per_class,
        'auc_macro': auc_macro
    }
    return metrics, text_report


In [ ]:
def load_and_preprocess_image(filepath):
  img = cv2.imread(filepath)  
  img = remove(img) 
  img = cv2.resize(img, (255, 255))  
  
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
  mean = np.mean(img, axis=(0, 1, 2), keepdims=True)
  std = np.std(img, axis=(0, 1, 2), keepdims=True)
  img = (img - mean) / (std + 1e-7)
  
  return img

def load_dataset(directory):
  images = []
  labels = []
  label_mapping = {'float32': 0, '0DS': 1, '1DS': 2, '2DS': 3}

  for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    img = load_and_preprocess_image(file_path)
    images.append(img)
    label = file.split('3DS')[0]
    labels.append(label_mapping[label])
  return np.array(images), np.array(labels)

In [ ]:
class_labels = ['0DS_', '1DS_', '2DS_', '3DS_']  


train_images, train_labels = load_dataset(str(TRAIN_DIR))
val_images,   val_labels   = load_dataset(str(VAL_DIR))
test_images,  test_labels  = load_dataset(str(TEST_DIR))


num_classes = 4
train_labels = to_categorical(train_labels, num_classes=num_classes)
val_labels   = to_categorical(val_labels,   num_classes=num_classes)
test_labels  = to_categorical(test_labels,  num_classes=num_classes)

print('0DS', train_images.shape, train_labels.shape)
print('1DS', val_images.shape,   val_labels.shape)
print('2DS', test_images.shape,  test_labels.shape)


In [ ]:
from tensorflow.keras.metrics import Recall

# Definir la arquitectura de la CNN #1

model = Sequential()
model.add(Flatten(input_shape=(255, 255, 3)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dense(4, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Softmax())

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossentropy(),
    metrics=['accuracy', Recall(name='recall')] 
)


history = model.fit(
    train_images,
    train_labels,
    epochs=50,
    validation_data=(test_images, test_labels), 
)


loss, accuracy, recall = model.evaluate(test_images, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')


epochs = range(1, len(history.history['loss']) + 1)
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_recall = history.history['recall']
val_recall = history.history['val_recall']


plt.figure(figsize=(18, 6))


plt.subplot(1, 3, 1)
plt.plot(epochs, train_loss, label='Train Loss', marker='o')
plt.plot(epochs, val_loss, label='Validation Loss', marker='o')
plt.title('Loss Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend()
plt.grid(True)


plt.subplot(1, 3, 2)
plt.plot(epochs, train_accuracy, label='Train Accuracy', marker='o')
plt.plot(epochs, val_accuracy, label='Validation Accuracy', marker='o')
plt.title('Accuracy Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend()
plt.grid(True)


plt.subplot(1, 3, 3)
plt.plot(epochs, train_recall, label='Train Recall', marker='o')
plt.plot(epochs, val_recall, label='Validation Recall', marker='o')
plt.title('Recall Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Recall', fontsize=12)
plt.legend()
plt.grid(True)


eval_text = f"Test Loss: {loss:.4f}\nTest Accuracy: {accuracy:.4f}\nTest Recall: {recall:.4f}"
plt.gcf().text(0.5, 0.02, eval_text, fontsize=12, ha='center', fontweight='bold')


plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.show()

In [ ]:
metrics1, report1 = evaluate_model_text(model, test_images, test_labels, model_name='Modelo 1')
display(metrics1['cm_df'])  
print(report1)              

In [ ]:
dir_1 = os.path.join(MODELS_DIR, "m1")
filename_1 = "weights_synthetic_m1.h5"
file_path_1 = os.path.join(dir_1, filename_1)
model.save(file_path_1)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
with open(dir_1 + "/weights_synthetic_m1.tflite","wb") as f:
    f.write(converter.convert())

In [ ]:
# Definir la arquitectura de la CNN #2

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(255, 255, 3))
base_model.trainable = False  

model2 = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

model2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', Recall(name='recall')]
)


history = model2.fit(
    train_images,
    train_labels,
    epochs=50,
    validation_data=(test_images, test_labels)
)


loss, accuracy, recall = model2.evaluate(test_images, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')


epochs = range(1, len(history.history['loss']) + 1)
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_recall = history.history['recall']
val_recall = history.history['val_recall']


plt.figure(figsize=(18, 6))


plt.subplot(1, 3, 1)
plt.plot(epochs, train_loss, label='Train Loss', marker='o')
plt.plot(epochs, val_loss, label='Validation Loss', marker='o')
plt.title('Loss Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend()
plt.grid(True)


plt.subplot(1, 3, 2)
plt.plot(epochs, train_accuracy, label='Train Accuracy', marker='o')
plt.plot(epochs, val_accuracy, label='Validation Accuracy', marker='o')
plt.title('Accuracy Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend()
plt.grid(True)


plt.subplot(1, 3, 3)
plt.plot(epochs, train_recall, label='Train Recall', marker='o')
plt.plot(epochs, val_recall, label='Validation Recall', marker='o')
plt.title('Recall Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Recall', fontsize=12)
plt.legend()
plt.grid(True)


eval_text = f"Test Loss: {loss:.4f}\nTest Accuracy: {accuracy:.4f}\nTest Recall: {recall:.4f}"
plt.gcf().text(0.5, 0.02, eval_text, fontsize=12, ha='center', fontweight='bold')


plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.show()

In [ ]:
metrics2, report2 = evaluate_model_text(model2, test_images, test_labels, model_name='Modelo 2')
display(metrics2['cm_df'])  
print(report2)              

In [ ]:
dir_2 = os.path.join(MODELS_DIR, "m2")
filename_2 = "weights_synthetic_m2.h5"
file_path_2 = os.path.join(dir_2, filename_2)
model.save(file_path_2)

converter = tf.lite.TFLiteConverter.from_keras_model(model2)
with open(dir_2 + "/weights_synthetic_m2.tflite","wb") as f:
    f.write(converter.convert())

In [ ]:
# Definir la arquitectura de la CNN #3

model3 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(255, 255, 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Recall(name='recall')])


history = model3.fit(train_images, train_labels, epochs=50, validation_data=(test_images, test_labels))


loss, accuracy, recall = model3.evaluate(test_images, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')


epochs = range(1, len(history.history['loss']) + 1)
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_recall = history.history['recall']
val_recall = history.history['val_recall']


plt.figure(figsize=(18, 6))


plt.subplot(1, 3, 1)
plt.plot(epochs, train_loss, label='Train Loss', marker='o')
plt.plot(epochs, val_loss, label='Validation Loss', marker='o')
plt.title('Loss Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend()
plt.grid(True)


plt.subplot(1, 3, 2)
plt.plot(epochs, train_accuracy, label='Train Accuracy', marker='o')
plt.plot(epochs, val_accuracy, label='Validation Accuracy', marker='o')
plt.title('Accuracy Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend()
plt.grid(True)


plt.subplot(1, 3, 3)
plt.plot(epochs, train_recall, label='Train Recall', marker='o')
plt.plot(epochs, val_recall, label='Validation Recall', marker='o')
plt.title('Recall Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Recall', fontsize=12)
plt.legend()
plt.grid(True)


eval_text = f"Test Loss: {loss:.4f}\nTest Accuracy: {accuracy:.4f}\nTest Recall: {recall:.4f}"
plt.gcf().text(0.5, 0.02, eval_text, fontsize=12, ha='center', fontweight='bold')


plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.show()

In [ ]:
metrics3, report3 = evaluate_model_text(model3, test_images, test_labels, model_name='Modelo 3')
display(metrics3['cm_df'])  
print(report3)              

In [ ]:
dir_3 = os.path.join(MODELS_DIR, "m3")
filename_3 = "weights_synthetic_m3.h5"
file_path_3 = os.path.join(dir_3, filename_3)
model.save(file_path_3)

converter = tf.lite.TFLiteConverter.from_keras_model(model3)
with open(dir_3 + "/weights_synthetic_m3.tflite","wb") as f:
    f.write(converter.convert())

In [ ]:
# Definir la arquitectura de la CNN #4
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

model4 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(255, 255, 3), kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(4, activation='softmax')
])

model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Recall(name='recall')])


history = model4.fit(train_images, train_labels, epochs=50, validation_data=(test_images, test_labels),  callbacks=[early_stopping])


loss, accuracy, recall = model4.evaluate(test_images, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')


epochs = range(1, len(history.history['loss']) + 1)
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_recall = history.history['recall']
val_recall = history.history['val_recall']


plt.figure(figsize=(18, 6))


plt.subplot(1, 3, 1)
plt.plot(epochs, train_loss, label='Train Loss', marker='o')
plt.plot(epochs, val_loss, label='Validation Loss', marker='o')
plt.title('Loss Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend()
plt.grid(True)


plt.subplot(1, 3, 2)
plt.plot(epochs, train_accuracy, label='Train Accuracy', marker='o')
plt.plot(epochs, val_accuracy, label='Validation Accuracy', marker='o')
plt.title('Accuracy Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend()
plt.grid(True)


plt.subplot(1, 3, 3)
plt.plot(epochs, train_recall, label='Train Recall', marker='o')
plt.plot(epochs, val_recall, label='Validation Recall', marker='o')
plt.title('Recall Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Recall', fontsize=12)
plt.legend()
plt.grid(True)


eval_text = f"Test Loss: {loss:.4f}\nTest Accuracy: {accuracy:.4f}\nTest Recall: {recall:.4f}"
plt.gcf().text(0.5, 0.02, eval_text, fontsize=12, ha='center', fontweight='bold')


plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.show()

In [ ]:
metrics4, report4 = evaluate_model_text(model4, test_images, test_labels, model_name='Modelo 4')
display(metrics4['cm_df'])  
print(report4)              

In [ ]:
dir_4 = os.path.join(MODELS_DIR, "m4")
filename_4 = "weights_synthetic_m4.h5"
file_path_4 = os.path.join(dir_4, filename_4)
model.save(file_path_4)

converter = tf.lite.TFLiteConverter.from_keras_model(model4)
with open(dir_4 + "/weights_synthetic_m4.tflite","wb") as f:
    f.write(converter.convert())

In [ ]:
# Definir la arquitectura de la CNN #5
from tensorflow.keras.layers import Input, Conv2D, Add, Activation, Flatten, Dense
from tensorflow.keras.models import Model

def residual_block(x, filters):
    shortcut = x
    x = Conv2D(filters, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

input_layer = Input(shape=(255, 255, 3))
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = residual_block(x, 32)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(4, activation='softmax')(x)

model5 = Model(inputs=input_layer, outputs=output_layer)
model5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Recall(name='recall')])

# Entrenar el modelo y capturar el historial
history = model5.fit(train_images, train_labels, epochs=50, validation_data=(test_images, test_labels))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy, recall = model5.evaluate(test_images, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')

# Extraer métricas del historial
epochs = range(1, len(history.history['loss']) + 1)
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_recall = history.history['recall']
val_recall = history.history['val_recall']

# Crear una figura con tres gráficos
plt.figure(figsize=(18, 6))

# Gráfica de Loss
plt.subplot(1, 3, 1)
plt.plot(epochs, train_loss, label='Train Loss', marker='o')
plt.plot(epochs, val_loss, label='Validation Loss', marker='o')
plt.title('Loss Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend()
plt.grid(True)

# Gráfica de Accuracy
plt.subplot(1, 3, 2)
plt.plot(epochs, train_accuracy, label='Train Accuracy', marker='o')
plt.plot(epochs, val_accuracy, label='Validation Accuracy', marker='o')
plt.title('Accuracy Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend()
plt.grid(True)

# Gráfica de Recall
plt.subplot(1, 3, 3)
plt.plot(epochs, train_recall, label='Train Recall', marker='o')
plt.plot(epochs, val_recall, label='Validation Recall', marker='o')
plt.title('Recall Over Epochs', fontsize=14)
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Recall', fontsize=12)
plt.legend()
plt.grid(True)

# Mostrar resultados finales en la parte inferior
eval_text = f"Test Loss: {loss:.4f}\nTest Accuracy: {accuracy:.4f}\nTest Recall: {recall:.4f}"
plt.gcf().text(0.5, 0.02, eval_text, fontsize=12, ha='center', fontweight='bold')

# Ajustar espaciado y mostrar las gráficas
plt.tight_layout(rect=[0, 0.05, 1, 0.95])
plt.show()

In [ ]:
metrics5, report5 = evaluate_model_text(model5, test_images, test_labels, model_name='Modelo 5')
display(metrics5['cm_df'])  
print(report5)              

In [ ]:
dir_5 = os.path.join(MODELS_DIR, "m5")
filename_5 = "weights_synthetic_m5.h5"
file_path_5 = os.path.join(dir_5, filename_5)
model.save(file_path_5)

converter = tf.lite.TFLiteConverter.from_keras_model(model5)
with open(dir_5 + "/weights_synthetic_m5.tflite","wb") as f:
    f.write(converter.convert())